# DesInventar Colombia Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process DesInventar Colombia disaster data and visualize it using interactive maps. We'll:

1. Download the DesInventar Colombia dataset
2. Convert the data to STAC items using pystac-monty
3. Display events on an interactive map
4. Allow selection of events to view related impacts
5. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [1]:
# Basic libraries
import tempfile

# Visualization libraries
import folium
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import requests

# STAC and pystac-monty
import shapely
from folium.plugins import MarkerCluster
from IPython.display import clear_output, display

from pystac_monty.extension import MontyExtension
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder
from pystac_monty.sources.desinventar import DesinventarDataSource, DesinventarTransformer

## 1. Download and Process DesInventar Colombia Data

First, let's download the DesInventar Colombia dataset and initialize the data source and transformer.

In [2]:
# Define DesInventar Colombia dataset URL
colombia_url = "https://www.desinventar.net/DesInventar/download/DI_export_col.zip"
country_code = "col"
iso3 = "COL"

# Download the dataset
print(f"Downloading DesInventar Colombia dataset from {colombia_url}...")
response = requests.get(colombia_url)
tmp_zip_file = tempfile.NamedTemporaryFile(suffix=".zip", delete=False)
tmp_zip_file.write(response.content)
print(f"Downloaded {len(response.content) / (1024*1024):.2f} MB")

# Initialize the data source and transformer
data_source = DesinventarDataSource(tmp_zip_file, country_code, iso3, colombia_url)
transformer = DesinventarTransformer(data_source)

# Initialize the geocoder
geocoder = WorldAdministrativeBoundariesGeocoder("../tests/data-files/world-administrative-boundaries.fgb")

Downloaded 15.77 MB


## 2. Create STAC Items from DesInventar Data

Now, let's transform the DesInventar data into STAC items.

In [3]:
# Create STAC items
print("Creating STAC items from DesInventar Colombia data...")
all_stac_items = transformer.get_items()
print(f"Created {len(all_stac_items)} STAC items")

Creating STAC items from DesInventar Colombia data...

NOTE: Removed 28684 duplicate items

NOTE: cannot map events for:
 {}
Created 95694 STAC items


In [ ]:
# Separate the STAC items by role
event_items = []
impact_items = []

for item in all_stac_items:
    # discard items older than 2020
    if item.datetime.year < 2020:
        continue
    roles = item.properties.get("roles", [])
    if "event" in roles:
        event_items.append(item)
    elif "impact" in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Impacts: {len(impact_items)}")

Events: 38997, Impacts: 56697


In [27]:
# Import STAC helper functions
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('.')
from stac_helpers import (
    add_items_to_collection,
    check_collection_exists,
    check_stac_api_availability,
    create_collection_fallback,
    create_collection_from_file,
    delete_collection,
)

# Define the STAC API endpoint
# Replace with your actual STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Define the collection IDs for each type of item
# These match the predefined collections in monty-stac-extension/examples
event_collection_id = "desinventar-events"
impact_collection_id = "desinventar-impacts"

# Define paths to the predefined collection definitions
event_collection_path = "../monty-stac-extension/examples/desinventar-events/desinventar-events.json"
impact_collection_path = "../monty-stac-extension/examples/desinventar-impacts/desinventar-impacts.json"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
STAC API is available at https://montandon-eoapi-stage.ifrc.org/stac


In [28]:
# Check if the collections exist and create them if they don't
if api_available:
    # Check and create event collection if needed
    delete_collection(stac_api_url, event_collection_id)
    event_collection_exists = check_collection_exists(stac_api_url, event_collection_id)
    if not event_collection_exists:
        print(f"\nAttempting to create collection '{event_collection_id}'...")
        event_collection_created = create_collection_from_file(stac_api_url, event_collection_path)
        if not event_collection_created:
            print("Trying fallback method to create event collection...")
            event_collection_created = create_collection_fallback(
                stac_api_url, 
                event_collection_id, 
                "USGS Earthquake events processed with pystac-monty",
                ["event", "source"]
            )
        event_collection_exists = event_collection_created
    
    # Check and create impact collection if needed
    delete_collection(stac_api_url, impact_collection_id)
    impact_collection_exists = check_collection_exists(stac_api_url, impact_collection_id)
    if not impact_collection_exists:
        print(f"\nAttempting to create collection '{impact_collection_id}'...")
        impact_collection_created = create_collection_from_file(stac_api_url, impact_collection_path)
        if not impact_collection_created:
            print("Trying fallback method to create impact collection...")
            impact_collection_created = create_collection_fallback(
                stac_api_url, 
                impact_collection_id, 
                "USGS Earthquake impacts processed with pystac-monty",
                ["impact", "source"]
            )
        impact_collection_exists = impact_collection_created
    
    if not (event_collection_exists and impact_collection_exists):
        print("\nWarning: One or more collections could not be created in the STAC API.")
        print("Some items may not be added to the STAC API.")
else:
    print("STAC API is not available. Skipping collection checks and creation.")

Collection 'desinventar-events' exists in the STAC API
Collection 'desinventar-events' deleted successfully
Collection 'desinventar-events' does not exist in the STAC API

Attempting to create collection 'desinventar-events'...
Collection 'desinventar-events' created successfully
Collection 'desinventar-impacts' exists in the STAC API
Collection 'desinventar-impacts' deleted successfully
Collection 'desinventar-impacts' does not exist in the STAC API

Attempting to create collection 'desinventar-impacts'...
Collection 'desinventar-impacts' created successfully


In [29]:
# Add the items to their respective collections if the API is available
if api_available:
    if event_collection_exists:
        print("Adding event items to the collection...")
        event_success, event_failed = add_items_to_collection(stac_api_url, event_collection_id, event_items, overwrite=True)
    else:
        print("Skipping adding event items because the collection doesn't exist")
        event_success, event_failed = 0, len(event_items)
    
    if impact_collection_exists:
        print("\nAdding impact items to the collection...")
        impact_success, impact_failed = add_items_to_collection(stac_api_url, impact_collection_id, impact_items)
    else:
        print("Skipping adding impact items because the collection doesn't exist")
        impact_success, impact_failed = 0, len(impact_items)
    
    total_success = event_success + impact_success
    total_failed = event_failed + impact_failed
    
    print(f"\nSummary: Added {total_success} items successfully, {total_failed} items failed")
else:
    print("Skipping adding items to collections because the API is not available")

Adding event items to the collection...
Processing batch 1 of 39 (1000 items)
Processing batch 2 of 39 (1000 items)
Processing batch 3 of 39 (1000 items)
Processing batch 4 of 39 (1000 items)
Failed to add item desinventar-event-COL-DGR-2008-00425. Status code: 424
Response: {"code":"DatabaseError","description":""}
Processing batch 5 of 39 (1000 items)
Failed to add item desinventar-event-COL-DGR-2008-01229. Status code: 424
Response: {"code":"DatabaseError","description":""}
Failed to add item desinventar-event-COL-DGR-2008-01296. Status code: 424
Response: {"code":"DatabaseError","description":""}
Failed to add item desinventar-event-COL-DGR-2008-01569. Status code: 424
Response: {"code":"DatabaseError","description":""}
Failed to add item desinventar-event-COL-DGR-2009-00182. Status code: 424
Response: {"code":"DatabaseError","description":""}
Processing batch 6 of 39 (1000 items)
Failed to add item desinventar-event-COL-DGR-2009-00677. Status code: 424
Response: {"code":"DatabaseE

KeyboardInterrupt: 